# CS3244 Machine Learning Team Project
## Prediction of the number of dengue cases in Singapore

First, we shall import necessary Python libraries.

In [1]:
# Import data processing tools
import pandas as pd
import numpy as np

import Levenshtein as lev

import csv, json, sys
from fuzzywuzzy import fuzz

# Import visualization tools
import matplotlib.pyplot as plt

# Import utility tools
import requests
from math import *
from datetime import datetime
from datetime import timedelta

# Import ML tools
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


We shall read from various CSV files to extract dataframe for each feature in the following cells:
* Weekly **dengue** cases (*dengue_df*)
* Annual **population** (*population_df*)
* Monthly mean **sunshine** hours (*sunshine_df*)
* Monthly **surface air temperature** (*surface_air_temperature_df*)

In [ ]:
# Read weekly dengue cases from csv files
raw_dengue_df = pd.read_csv(
    "Datasets/Dengue Cluster Data/weekly-number-of-dengue-and-dengue-haemorrhagic-fever-cases.csv")

# Clean data by filtering by type_dengue and remove the redundant column
dengue_df = raw_dengue_df[raw_dengue_df['type_dengue'].map(
    lambda x: str(x) == "Dengue")].drop('type_dengue', 1).reset_index()

# Rename eweek to week
dengue_df = dengue_df.rename(columns={"eweek": "week"}, errors="raise")
# Display first 5 rows of the dataframe
dengue_df.head()

In [ ]:
# Read from CSV
raw_population_df = pd.read_csv(
    "Datasets/Population Data/singapore-residents-by-age-group-ethnic-group-and-sex-end-june-annual.csv")
# Only get data from 2014 to 2018
population_df = raw_population_df[raw_population_df['year'].map(
    lambda x: 2014 <= int(x) <= 2018)]
# Only get total residents for each year
population_df = population_df[population_df['level_1'].map(
    lambda x: str(x) == 'Total Residents')].drop('level_1', 1).drop('level_2', 1)
population_df = population_df.astype({'year' : 'int32', 'value': 'int32'})
population_df = population_df.groupby('year').sum()
population_df = population_df.rename(columns={"value": "population"}, errors="raise")
# Display dataframe
population_df.head()

In [ ]:
# Read from CSV
raw_sunshine_df = pd.read_csv("Datasets/Sunshine Data/sunshine-duration-monthly-mean-daily-duration.csv")
# Split year_month into year and month column
raw_sunshine_df = raw_sunshine_df.rename(columns={"month": "year_month"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
for i in range(len(raw_sunshine_df)):
    year_col[i] = int(raw_sunshine_df['year_month'][i].split('-')[0])
    month_col[i] = int(raw_sunshine_df['year_month'][i].split('-')[1])
raw_sunshine_df.insert(1, 'year', year_col)
raw_sunshine_df.insert(2, 'month', month_col)
sunshine_df = raw_sunshine_df.drop('year_month', 1)
# Display
sunshine_df.head()

In [ ]:
# Read from CSV
raw_temperature_df = pd.read_csv(
    "Datasets/Surface Air Temperature Data/surface-air-temperature-monthly-mean-daily-maximum.csv")
# Split year_month into year and month column
raw_temperature_df = raw_temperature_df.rename(columns={"month": "year_month"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
for i in range(len(raw_temperature_df)):
    year_col[i] = int(raw_temperature_df['year_month'][i].split('-')[0])
    month_col[i] = int(raw_temperature_df['year_month'][i].split('-')[1])
raw_temperature_df.insert(1, 'year', year_col)
raw_temperature_df.insert(2, 'month', month_col)
surface_air_temperature_df = raw_temperature_df.drop('year_month', 1)
# Display
surface_air_temperature_df.head()

Next, we shall merge the dataframes into one general dataframe **df** based on *year*, *month* and *week*.

In [ ]:
# Merge dengue_df and population_df by year
df = dengue_df.merge(population_df, left_on='year', right_on='year').drop('index', 1)

# Create new column for cases_per_capita = number / population
# Create new column for month = (datetime(year, 1, 1) + timedelta(days=week*7)).momth
cases_per_capita = pd.Series([], dtype='float64')
month = pd.Series([], dtype='int32')
for i in range(len(df)):
    if isnan(df['number'][i]):
        cases_per_capita[i] = 0
    else:
        cases_per_capita[i] = int(df['number'][i]) / int(df['population'][i])
    year = int(df['year'][i])
    week = int(df['week'][i])
    if week > 52:
        month[i] = 12
    else:
        month[i] = (datetime(year, 1, 1) + timedelta(days=7*week)).month
df.insert(4, 'cases_per_capita', cases_per_capita)
df.insert(1, 'month', month)

# Merge sunshine and surface air temperature dataframe
df = pd.merge(df, sunshine_df, on=['year', 'month'])
df = pd.merge(df, surface_air_temperature_df, on=['year', 'month'])

# Display dataframe
df.head()

Now, we scrape the weather data from 2014 to 2019 for 5 weather stations:

In [ ]:
#weather station
# Scrape from websites
base_url = "http://www.weather.gov.sg/files/dailydata/DAILYDATA_" #S24_201911.csv

# (East) Changi = 24, (West) Tuas South = 115, (North) Khatib = 122, (South) Marina Barrage = 108
# (Central) Ang Mo Kio = 109
station_dict = {24: 'Changi', 115: 'Tuas South', 122: 'Khatib', 108: 'Marina Barrage', 109: 'Ang Mo Kio'}
stations_list = [24, 115, 122, 108, 109]

# DataFrame template
df_template = pd.DataFrame(columns = ['Station', 'Year', 'Month', 'Day', 'Daily Rainfall Total (mm)',
       'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
       'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'])
weather_df = df_template.copy()

# Scrape stations and combine into a single csv file
for station in stations_list:
    station_string = "S"+str(station) if (station>9) else "S0"+str(station)
    station_df = df_template.copy()
    for year in range(2014,2020):
        for month in range(1,13):
            month_string = str(month) if (month>9) else "0"+str(month)
            url = base_url+station_string+"_"+str(year)+month_string+".csv"
            # Get csv files to temp files, and store into DataFrame
            try:
                r = requests.get(url, allow_redirects=True)
                open('temp.csv', 'wb').write(r.content)
                station_df = station_df.append(pd.read_csv("temp.csv", encoding = "ISO-8859-1"))
                weather_df = weather_df.append(pd.read_csv("temp.csv", encoding = "ISO-8859-1"))
            except:
                print(url)
                continue
    # Store into weather data folder
    csv_filename = 'Datasets/Weather Data/' + station_dict[station] + '.csv'
    station_df.to_csv(csv_filename, index=False)


We then add the week column to the weather data

In [ ]:
#Weather station 
station_dict = {24: 'Changi', 115: 'Tuas South', 122: 'Khatib', 108: 'Marina Barrage', 109: 'Ang Mo Kio'}
stations_list = [24, 115, 122, 108, 109]

#add week column to each weather stations csv
for station in stations_list:
    week_col = pd.Series([], dtype='int32')
    df = pd.read_csv("Datasets/Weather Data/" + station_dict[station] + ".csv")
    for i in range(len(df)):
        week_col[i] = datetime(int(df['Year'][i]), int(df['Month'][i]), int(df['Day'][i])).isocalendar()[1]
    df.insert(3, 'Week', week_col)
    csv_filename = 'Datasets/Weather Data/' + station_dict[station] + '.csv'
    df.to_csv(csv_filename, index=False)

Convert PSI data to contain year, month, week and day columns:

In [ ]:
#PSI
raw_psi_df = pd.read_csv("Datasets/PSI Data/psi_df_2016_2019.csv")

#split to year month and week
raw_psi_df = raw_psi_df.rename(columns={"timestamp": "year_month_week_day"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
day_col = pd.Series([], dtype='int32')
week_col = pd.Series([], dtype='int32')
for i in range(len(raw_psi_df)):
    year_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[0])
    month_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[1])
    day_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[2][0:2])
    week_col[i] = datetime(int(year_col[i]), int(month_col[i]), int(day_col[i])).isocalendar()[1]
raw_psi_df.insert(0, 'year', year_col)
raw_psi_df.insert(1, 'month', month_col)
raw_psi_df.insert(2, 'week', week_col)
raw_psi_df.insert(3, 'day', day_col)
psi_df = raw_psi_df.drop(['year_month_week_day'], 1)

#display
psi_df.head()


Merge 2014-2019 location dengue cluster into one csv
Add year, month, week and day columns
Split locality with '/' and '('
save as new csv

In [5]:
#merge the location data into one csv
raw_location_dengue_2014_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2014-Dengue-Cluster.csv")
raw_location_dengue_2015_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2015-Dengue-Cluster.csv")
raw_location_dengue_2016_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2016-Dengue-Cluster.csv")
raw_location_dengue_2017_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2017-Dengue-Cluster.csv", encoding='unicode_escape')
raw_location_dengue_2018_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2018-Dengue-Cluster.csv")
raw_location_dengue_2019_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2019-Dengue-Cluster.csv")

raw_location_dengue_df = raw_location_dengue_2014_df.append([raw_location_dengue_2015_df, raw_location_dengue_2016_df, raw_location_dengue_2017_df,
         raw_location_dengue_2018_df, raw_location_dengue_2019_df])

#save the raw csv
csv_filename = 'Datasets/Dengue Location Cluster Data/' + 'raw_location_dengue' + '.csv'
raw_location_dengue_df.to_csv(csv_filename, index=False)

raw_location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/raw_location_dengue.csv")

#split year_month_day to year month and week columns
raw_location_dengue_df = raw_location_dengue_df.rename(columns={"date": "year_month_week_day"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
day_col = pd.Series([], dtype='int32')
week_col = pd.Series([], dtype='int32')
locality_col = pd.Series([], dtype='str')
# shortcut_list = ['rd', 'jln', 'lor', 'dr', 'ave', 'st', 'pk', 'gdns', 'nth', "s'goon", 'ter']
# shortcut_dict = {'rd': 'Road', 'jln': 'Jalan', 'lor': 'Lorong', 'dr': 'Drive', 'ave': 'Avenue',
#                 'st': 'Street', 'pk' : 'Park', 'gdns' : 'Gardens', 'nth' : 'North', "s'goon" : 'Serangoon',
#                 'ter' : 'Terminal'}

for i in range(len(raw_location_dengue_df)):
#     address = []
    year_col[i] = 2000 + int(str(raw_location_dengue_df['year_month_week_day'][i])[0:2])
    month_col[i] = int(str(raw_location_dengue_df['year_month_week_day'][i])[2:4])
    day_col[i] = int(str(raw_location_dengue_df['year_month_week_day'][i])[4:])
    week_col[i] = datetime(int(year_col[i]), int(month_col[i]), int(day_col[i])).isocalendar()[1]
    #split location with '/'
    locality_col[i] = raw_location_dengue_df['locality'][i].split('/')[0]
    #split location with '('
    locality_col[i] = locality_col[i].split('(')[0]
    locality_col[i] = "".join([x if ord(x) < 128 else ' ' for x in locality_col[i]])
#     address = locality_col[i].split(' ')
#     for j in range(len(address)):
#         for shortcut in shortcut_list:
#             if(str.lower(address[j]) == shortcut):
#                 address[j] = shortcut_dict[shortcut]
#                 break
#     print(address)
#     locality_col[i] = address[0]
#     print(locality_col[i])
#     for j in range(len(address)):
#         if j != 0 and address[j] != '':
#             locality_col[i] += " " + address[j]
    
        
raw_location_dengue_df.insert(1, 'year', year_col)
raw_location_dengue_df.insert(2, 'month', month_col)
raw_location_dengue_df.insert(3, 'week', week_col)
raw_location_dengue_df.insert(4, 'day', day_col)
raw_location_dengue_df.insert(5, 'address', locality_col)
location_dengue_df = raw_location_dengue_df.drop(['year_month_week_day', 'locality'], 1)

#save
csv_filename = 'Datasets/Dengue Location Cluster Data/' + 'location_dengue' + '.csv'
location_dengue_df.to_csv(csv_filename, index=False)



Convert the buildings.json to a csv file (buildings.json contain all addresses in Singapore)

In [ ]:
#path to json file
# df = pd.read_json("Datasets/buildings.json", orient="records")

# df.head()

fileInput = 'Datasets/buildings.json'
fileOutput = 'Datasets/buildings.csv'

inputFile = open(fileInput)
outputFile = open(fileOutput, 'w')
data = json.load(inputFile)
inputFile.close()

output = csv.writer(outputFile)
output.writerow(data[0].keys()) 

for row in data:
    output.writerow(row.values())

 

Find unique locations in the location_dengue cluster csv

In [6]:
#csv of unique locations in dengue location cluster (2014 to 2019)
location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue.csv")
location_set = set()
for i in range(len(location_dengue_df)):
    location_set.add(location_dengue_df['address'][i])

fileOutput = 'Datasets/unique_locations.csv'
outputFile = open(fileOutput, 'w')
output = csv.writer(outputFile)
output.writerow(['unique location', 'X', 'Y'])
for location in location_set:
    output.writerow([location])
    

In [ ]:
#add x,y coordinates to the unique locations
unique_location_df = pd.read_csv("Datasets/unique_locations.csv")
open_buildings_df = pd.read_csv("Datasets/buildings.csv")
buildings_df = open_buildings_df.drop(['POSTAL'], 1)
for i in range(len(unique_location_df)):
    for j in range(len(buildings_df)):
        if str.upper(str(unique_location_df['unique location'][i])) in str(buildings_df['ADDRESS'][j]):
            unique_location_df['X'][i] = buildings_df['X'][j]
            unique_location_df['Y'][i] = buildings_df['Y'][j]
            break

csv_filename = "Datasets/unique_locations.csv"            
unique_location_df.to_csv(csv_filename, index=False)

In [ ]:
#add x,y coordinates to the unique locations
unique_location_df = pd.read_csv("Datasets/unique_locations.csv")
open_buildings_df = pd.read_csv("Datasets/buildings.csv")
buildings_df = open_buildings_df.drop(['POSTAL'], 1)
count = 0
locality_col = pd.Series([], dtype='str')

for i in range(len(unique_location_df)):
    for j in range(len(buildings_df)):
        if fuzz.token_set_ratio(str(unique_location_df['unique location'][i]), str(buildings_df['ADDRESS'][j])) >= 60:
            count += 1
            unique_location_df['X'][i] = buildings_df['X'][j]
            unique_location_df['Y'][i] = buildings_df['Y'][j]
            locality_col[i] = buildings_df['ADDRESS'][j]
            break

print(count)
csv_filename = "Datasets/unique_locations_fuzzywuzzy.csv"
unique_location_df.insert(3, 'address from buildings', locality_col)
unique_location_df.to_csv(csv_filename, index=False)

Next, we can visualize the data.

In [ ]:
# Copy the dataframe, not to corrupt the original
vis_df = df.copy()
x_data = range(0, vis_df.shape[0])

# columns to visualize
columns = ['cases_per_capita', 'mean_sunshine_hrs', 'temp_mean_daily_max']

# plot line graphs
for column in columns:
    f = plt.figure()
    ax = f.add_subplot()
    ax.plot(x_data, vis_df[column], label=column)
    ax.legend()

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

